In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/deeptweetsdataset/sample_submission.csv
/kaggle/input/deeptweetsdataset/training.csv
/kaggle/input/deeptweetsdataset/train.csv
/kaggle/input/deeptweetsdataset/test.csv


# **Imports used throughout this task**

In [37]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import scipy as sp
import numpy as np
from sklearn.metrics import accuracy_score
import string
import nltk
nltk.download("omw-1.4")

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# TweetText Preprocessing

In [38]:
def preprocess(text):
    
    punctuation = string.punctuation
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # remove urls
    text = re.sub(r'https?://\S+', '', text, flags=re.MULTILINE)
    
    # remove punctuations and usernames and lemmatization
    table = str.maketrans('', '', punctuation)
    words = [w.translate(table).lower() for w in text.split(" ")]    
    
    # remove usernames and tags and stopwords
    words = [w for w in words if len(w)>=1 and w[0] is not "@" and w[0] is not "#" and w not in stop_words]
    words = set(words)
    
    # lemmatization
    words = [lemmatizer.lemmatize(w) for w in words if len(w)>1]
    
    return words
    

In [39]:

train =  pd.read_csv("/kaggle/input/deeptweetsdataset/train.csv")
test =  pd.read_csv("/kaggle/input/deeptweetsdataset/test.csv")

In [40]:
train["TweetText"] = train["TweetText"].apply(lambda x: preprocess(x))
print(train)

                 TweetId     Label  \
0     304271250237304833  Politics   
1     304834304222064640  Politics   
2     303568995880144898    Sports   
3     304366580664528896    Sports   
4     296770931098009601    Sports   
...                  ...       ...   
6520  296675082267410433  Politics   
6521  306677536195231746    Sports   
6522  306451295307431937    Sports   
6523  306088574221176832    Sports   
6524  277090953242759169  Politics   

                                              TweetText  
0     [usaid, value, seckerry, value, term, dollar, ...  
1                                    [fear, rraina1481]  
2     [indie, wwc13, video, watch, final, highlight,...  
3     [chadwickmodels, rt, albertpark, ausgrandprix,...  
4     [cricketfox, thing, good, feedback, thanks, al...  
...                                                 ...  
6520  [wreath, pm, monument, photo, martyr, algiers,...  
6521  [chocolate, crumbling, secret, middle, chennai...  
6522               [i

In [41]:
# Load the training data
test['TweetText'] = test['TweetText'].apply(lambda x: preprocess(x))
print(test['TweetText'])

0       [side, end, wide, threaten, get, mason, stab, ...
1       [pound, thx, avoirdupois, mass, mrbrown, deriv...
2       [sea, shore, construction, along, sochi2014, b...
3       [seckerryu2019s, meeting, remark, transcript, ...
4       [go, base, usd, price, hammer, go, first, ipla...
                              ...                        
2605    [ground, amp, qualifier, runnersup, game, 2012...
2606    [reesedward, mission, hi, mission, here, 16, p...
2607    [purchase, sunrisersipl, first, iplauction, pe...
2608    [timkaine, seckerry, statedept4us, charlottesv...
2609    [account, bill, pinterest, lfc, staff, here, s...
Name: TweetText, Length: 2610, dtype: object


# Features Extraction

In [42]:
train_words_count={}

for i in range(len(train)):
    for j in train['TweetText'][i]:
        if j in train_words_count.keys():
            train_words_count[j]+=1
        else:
            train_words_count[j]=1
train_words_count

{'usaid': 5,
 'value': 9,
 'seckerry': 51,
 'term': 19,
 'dollar': 9,
 'measured': 2,
 'deepest': 2,
 'statedept': 10,
 'american': 65,
 'fear': 8,
 'rraina1481': 4,
 'indie': 16,
 'wwc13': 30,
 'video': 181,
 'watch': 149,
 'final': 159,
 'highlight': 31,
 'australia': 105,
 'west': 47,
 'chadwickmodels': 1,
 'rt': 873,
 'albertpark': 8,
 'ausgrandprix': 45,
 'chelscanlan': 12,
 'nitro': 2,
 'circus': 2,
 'cantwaitforausgp': 29,
 'theymakeitlooksoeasy': 1,
 'cricketfox': 1,
 'thing': 41,
 'good': 103,
 'feedback': 28,
 'thanks': 140,
 'always': 26,
 'rajan': 11,
 'policy': 52,
 'space': 11,
 'dr': 27,
 'consolidation': 4,
 'fiscal': 19,
 'monetary': 4,
 'act': 35,
 'create': 28,
 'fact': 35,
 'doesnt': 19,
 'forced': 3,
 'take': 151,
 'jobsnotcuts': 11,
 'leave': 12,
 'unpaid': 1,
 'defense': 12,
 'cut': 57,
 'congress': 54,
 'impending': 2,
 'spending': 19,
 '800000': 2,
 'employee': 6,
 'stop': 25,
 'clarke': 55,
 'wkt': 10,
 'bcci': 199,
 'run': 269,
 '1st': 237,
 '39': 10,
 'au': 

In [43]:
test_words_count={}
for i in range(len(test)):
    for j in test['TweetText'][i]:
        if j in test_words_count.keys():
            test_words_count[j]+=1
        else:
            test_words_count[j]=1
test_words_count

{'side': 14,
 'end': 29,
 'wide': 5,
 'threaten': 2,
 'get': 57,
 'mason': 3,
 'stab': 1,
 'yard': 3,
 'long': 11,
 'throw': 2,
 '28': 8,
 'bennett': 3,
 'home': 17,
 'pound': 2,
 'thx': 1,
 'avoirdupois': 1,
 'mass': 3,
 'mrbrown': 5,
 'derives': 1,
 'abbreviation': 1,
 'asking': 2,
 'series': 12,
 'unit': 1,
 'see': 45,
 'aulia': 1,
 'sea': 1,
 'shore': 1,
 'construction': 3,
 'along': 3,
 'sochi2014': 24,
 'black': 3,
 'seckerryu2019s': 5,
 'meeting': 31,
 'remark': 18,
 'transcript': 8,
 'video': 55,
 'minister': 35,
 'westerwelle': 4,
 'foreign': 39,
 'go': 72,
 'base': 14,
 'usd': 1,
 'price': 17,
 'hammer': 4,
 'first': 81,
 'iplauction': 127,
 'player': 47,
 'mumbai': 2,
 'indian': 33,
 'ponting': 3,
 '400000': 4,
 'ricky': 2,
 'begun': 2,
 'draw': 6,
 'caruana': 3,
 'zurich': 3,
 'viswanathan': 3,
 'anand': 4,
 'fabiano': 3,
 'tonight': 40,
 'say': 67,
 'tweet': 25,
 '81892': 1,
 'lfctvliverpoolfccom': 1,
 'game': 48,
 'u': 107,
 'email': 7,
 'send': 13,
 'kop': 15,
 'start': 

In [44]:
for i in train_words_count.keys():
    if i not in test_words_count:
        test_words_count[i] = 0
test_words_count

{'side': 14,
 'end': 29,
 'wide': 5,
 'threaten': 2,
 'get': 57,
 'mason': 3,
 'stab': 1,
 'yard': 3,
 'long': 11,
 'throw': 2,
 '28': 8,
 'bennett': 3,
 'home': 17,
 'pound': 2,
 'thx': 1,
 'avoirdupois': 1,
 'mass': 3,
 'mrbrown': 5,
 'derives': 1,
 'abbreviation': 1,
 'asking': 2,
 'series': 12,
 'unit': 1,
 'see': 45,
 'aulia': 1,
 'sea': 1,
 'shore': 1,
 'construction': 3,
 'along': 3,
 'sochi2014': 24,
 'black': 3,
 'seckerryu2019s': 5,
 'meeting': 31,
 'remark': 18,
 'transcript': 8,
 'video': 55,
 'minister': 35,
 'westerwelle': 4,
 'foreign': 39,
 'go': 72,
 'base': 14,
 'usd': 1,
 'price': 17,
 'hammer': 4,
 'first': 81,
 'iplauction': 127,
 'player': 47,
 'mumbai': 2,
 'indian': 33,
 'ponting': 3,
 '400000': 4,
 'ricky': 2,
 'begun': 2,
 'draw': 6,
 'caruana': 3,
 'zurich': 3,
 'viswanathan': 3,
 'anand': 4,
 'fabiano': 3,
 'tonight': 40,
 'say': 67,
 'tweet': 25,
 '81892': 1,
 'lfctvliverpoolfccom': 1,
 'game': 48,
 'u': 107,
 'email': 7,
 'send': 13,
 'kop': 15,
 'start': 

In [45]:
# add missing words from train data present in test data
for i in test_words_count.keys():
    if i not in train_words_count:
        train_words_count[i] = 0

In [46]:
for i in train_words_count.keys():
    train[i] = 0

# Set values in the new columns
for i in range(len(train)):
    for j in train['TweetText'][i]:
        train.loc[i, j] = 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [47]:
for i in test_words_count.keys():
    test[i] = 0

# Set values in the new columns
for i in range(len(test)):
    for j in test['TweetText'][i]:
        test.loc[i, j] = 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [48]:
train.drop(['TweetId', 'TweetText'], axis=1, inplace=True)
test.drop(['TweetId', 'TweetText'], axis=1, inplace=True)

In [49]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder().fit(train["Label"].tolist())
y = label_encoder.transform(train["Label"].tolist())
y

array([0, 0, 1, ..., 1, 1, 0])

In [50]:
X = train.drop(['Label'], axis=1)
X.head()

,usaid,value,seckerry,term,dollar,measured,deepest,statedept,american,fear,...,soturelated,lazio,preserved,twostate,eliminator,reesedward,timkaine,charlottesville,reproberthurt,shankly
0,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Model Training

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.230)

In [52]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
clf.score(X_train, y_train)


0.9982085987261147

In [53]:
accuracy = clf.score(X_train, y_train)
print(f"Accuracy on validation set: {accuracy:.2f}")

Accuracy on validation set: 1.00


In [54]:
from sklearn.metrics import classification_report
pred = clf.predict(X_test)
print(classification_report(y_test, pred, target_names=("Politics", "Sports")))

              precision    recall  f1-score   support

    Politics       0.95      0.94      0.95       730
      Sports       0.94      0.96      0.95       771

    accuracy                           0.95      1501
   macro avg       0.95      0.95      0.95      1501
weighted avg       0.95      0.95      0.95      1501



# Model Testing - Prediction 

In [55]:
def classify(label):
    
    if label == 0:
        return "Politics" 
    return "Sports"


test_data = pd.read_csv("/kaggle/input/deeptweetsdataset/test.csv")


test_data['Label']=[classify(label) for label in clf.predict(test)]
test_data

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,TweetId,TweetText,Label
0,306486520121012224,'28. The home side threaten again through Maso...,Politics
1,286353402605228032,'@mrbrown @aulia Thx for asking. See http://t....,Sports
2,289531046037438464,'@Sochi2014 construction along the shores of t...,Sports
3,306451661403062273,'#SecKerry\u2019s remarks after meeting with F...,Politics
4,297941800658812928,'The #IPLauction has begun. Ricky Ponting is t...,Politics
...,...,...,...
2605,282023761044189184,'Qualifier 1 and Eliminator games will be play...,Sports
2606,303879735006601216,"@reesedward Hi Edward, it's not a #peacekeepin...",Sports
2607,297956846046703616,'Perera was @SunRisersIPL first #IPL purchase ...,Sports
2608,304265049537658880,"'#SecKerry: Thanks to Senator @TimKaine, @RepR...",Sports
